In [1]:
from art_api import config

In [5]:
from pathlib import Path  

BUCKET_NAME = 'art-api'
#BUCKET_TRAIN_DATA_PATH = 'data/train_1k.csv'
BUCKET_TRAIN_DATA_PATH = 'data/raw_data'
BUCKET_TRAIN_DATA_FILE = 'df_yourpaintings.csv'
PATH = Path("../raw_data")
PATH_YOURPAINTINGS = PATH/"yourpaintings"
PATH_YOURPAINTINGS_SM = PATH/"yourpaintings_sm"
PATH_BING = PATH/"bing"
MODEL_NAME = 'art_api'
MODEL_VERSION = 'baseline'
PATH_FILE = Path(PATH_YOURPAINTINGS/"df_yourpaintings.csv")  

In [6]:
from bing_image_downloader import downloader

dir = PATH_BING
downloader.download('bird painting', limit = 3500, output_dir = dir, adult_filter_off = False)

[%] Downloading Images to /home/erniesg/code/erniesg/art_api/notebooks/../raw_data/bing/bird painting


[!!]Indexing page: 1

[%] Indexed 9 Images on Page 1.


[%] Downloading Image #1 from https://i.pinimg.com/736x/3c/f9/53/3cf953f1beacd226e64c327d9f10e6f2.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://i.pinimg.com/736x/e5/99/24/e5992452d7d8e734992aa635dc334e69.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://www.acstickley.com/wp-content/uploads/2017/04/WilliamAdams2.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://i.pinimg.com/736x/77/a9/3d/77a93d02799e677d3f0649567b02c63c.jpg
[%] File Downloaded !

[%] Downloading Image #5 from http://www.wallpapers13.com/wp-content/uploads/2016/02/Swan-bird-lake-beautiful-sunset-wallpaper-hd-1600x1200.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://www.puttyandpaint.com/images/uploads/artistworks/26211/cache/final-my-4__sized_l.jpg
[%] File Downloaded !

[%] Downloading Image #7 fr

KeyboardInterrupt: 

In [ ]:
downloader.download('cheetah', limit = 20, output_dir = dir, adult_filter_off = False)

In [5]:
import requests
from bs4 import BeautifulSoup
import json
import re
from urllib.parse import urlparse, quote, quote_plus
import io
import time
import os
import threading
import argparse

headers = {      
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Linux; Android 12) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Mobile Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-GB,en;q=0.9",
}


all_images = {}

threads_running = 0 # don't change this

def get_extension(url):
    path = urlparse(url).path.strip('/')
    path,filename = os.path.split(path)
    filename,ext = os.path.splitext(filename)
    if ext:
        return ext
    else:
        return '.jpg'
    

def threaded_download(savepath,url):
    global threads_running   
    
    try:
        r = requests.get(url,headers=headers)
    except Exception as e:
        print("Request Error : "+url)
        threads_running-=1
        return
        
    if r.status_code==200:
        if 'image' in r.headers.get('content-type','').lower():
            with open(savepath,'wb',) as f:
                f.write(r.content)
            print('Downloaded '+os.path.basename(savepath))  
        else:
            print("Not an Image : "+url)
            
    else:
        print(f"{r.status_code} Error : "+url+" ")
       
    threads_running-=1 


def collect_images_from_google(query):
    chunk = 0
    step = 0
    prev_len = 0
    no_more = 0
    
    while True:
        r = requests.get(f'https://www.google.com/search?q={query}&tbm=isch&biw=1271&bih=697&async=_id:islrg_c,_fmt:json&asearch=ichunklite&ved=0ahUKEwjw_KTXkM_3AhVJxDgGHR-pDeoQtDIIPSgA&start={chunk}&ijn={step}',headers=headers)
        
        if r.status_code==200:
            json_text = r.content.decode('utf8')
            #json_text = r.content.decode('utf8').removeprefix(")]}'")
            json_data = json.loads(json_text)
            try:
                results = json_data['ichunklite']['results']
                for result in results:
                    original_image = result['viewer_metadata']['original_image']['url']
                    all_images[original_image] = result['image_docid']
            except:
                pass
                
            chunk +=100
            step +=1   
        
        if no_more>5:
            break
        if prev_len==len(all_images):
            no_more+=1
        else:
            print(len(all_images))
            prev_len = len(all_images)
            no_more = 0

def download_images(query):   
    global threads_running
    print("Downloading "+str(len(all_images))+" images")
    if not os.path.exists(os.path.join("Images",f"{query}")):
        os.makedirs(os.path.join("Images",f"{query}"))
      
    for image_link,doc_id in all_images.items():  
        image_save_path = os.path.join("Images",f"{query}",doc_id+get_extension(image_link))   
        if os.path.exists(image_save_path):
            if os.path.getsize(image_save_path)>1:
                continue     
        threading.Thread(target=threaded_download,kwargs={'url':image_link,'savepath':image_save_path}).start()  
        threads_running+=1
        while threads_running>9:
            time.sleep(1)

    while threads_running>0:
        time.sleep(1)


# if __name__=="__main__":
#     parser = argparse.ArgumentParser(description = "Download images from google")
#     parser.add_argument("-k", "--keyword", help = "Example: Help argument", required = True, default = "")
#     arguments = parser.parse_args()
#     keyword= arguments.keyword
    
#     collect_images_from_google(keyword)
#     download_images(keyword)

In [6]:
collect_images_from_google("cat")
download_images("cat")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import selenium
from selenium import webdriver
import requests
import shutil
import hashlib
import os
import io
import time
from PIL import Image

# Define the path to chrome driver
DRIVER_PATH = '/usr/local/bin/chromedriver' # e.g. 'C:\\Users\\Username\\chromedriver_win32\\chromedriver'
wd = webdriver.Chrome(executable_path = DRIVER_PATH)
# This searches images from google.com
wd.get('https://google.com')

# Create the scraper class
class GoogleScraper():
    '''Downloades images from google based on the query.
       webdriver - Selenium webdriver
       max_num_of_images - Maximum number of images that we want to download
    '''
    def __init__(self, webdriver:webdriver, max_num_of_images:int):
        self.wd = webdriver
        self.max_num_of_images = max_num_of_images

    def _scroll_to_the_end(self):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  

    def _build_query(self, query:str):
        return f"https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={query}&oq={query}&gs_l=img"

    def _get_info(self, query: str):
        image_urls = set()

        wd.get(self._build_query(query))
        self._scroll_to_the_end()

        # img.Q4LuWd is the google tumbnail selector
        thumbnails = self.wd.find_elements_by_css_selector("img.Q4LuWd")

        print(f"Found {len(thumbnails)} images...")
        print(f"Getting the links...")

        for img in thumbnails[0:self.max_num_of_images]:
            # We need to click every thumbnail so we can get the full image.
            try:
                img.click()
            except Exception:
                print('ERROR: Cannot click on the image.')
                continue

            images = wd.find_elements_by_css_selector('img.n3VNCb')
            time.sleep(0.3)

            for image in images:
                if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                    image_urls.add(image.get_attribute('src'))

        return image_urls

    def download_image(self, folder_path:str, url:str):
        try:
            image_content = requests.get(url).content

        except Exception as e:
            print(f"ERROR: Could not download {url} - {e}")

        try:
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file).convert('RGB')
            file = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')

            with open(file, 'wb') as f:
                image.save(f, "JPEG", quality=85)
            print(f"SUCCESS: saved {url} - as {file}")

        except Exception as e:
            print(f"ERROR: Could not save {url} - {e}")

    def scrape_images(self, query:str, folder_path= 'path'):
        folder = os.path.join(folder_path,'_'.join(query.lower().split(' ')))

        if not os.path.exists(folder):
            os.makedirs(folder)

        image_info = self._get_info(query)
        print(f"Downloading images...")

        for image in image_info:
            self.download_image(folder, image)

# Run scrape command
gs = GoogleScraper(wd, 100)
gs.scrape_images('fufu meal')

In [7]:
from art_api import config

In [8]:
config.PATH_YOURPAINTINGS_SM

PosixPath('../raw_data/yourpaintings_sm')

In [62]:
import bs4
import requests
from selenium import webdriver
import os
import time

#creating a directory to save images
folder_name = 'images'
if not os.path.isdir(folder_name):
    os.makedirs(folder_name)

def download_image(url, folder_name, num):

    # write image to file
    reponse = requests.get(url)
    if reponse.status_code==200:
        with open(os.path.join(folder_name, str(num)+".jpg"), 'wb') as file:
            file.write(reponse.content)



chromePath=r'C:\Users\net51\Documents\MyPythonScripts\Drivers\chromedriver.exe'
driver=webdriver.Chrome(chromePath)

search_URL = "https://www.google.com/search?q=car+parts&source=lnms&tbm=isch"
driver.get(search_URL)

#//*[@id="islrg"]/div[1]/div[1]
#//*[@id="islrg"]/div[1]/div[50]
#//*[@id="islrg"]/div[1]/div[25]
#//*[@id="islrg"]/div[1]/div[75]
#//*[@id="islrg"]/div[1]/div[350]


a = input("Waiting...")

#Scrolling all the way up
driver.execute_script("window.scrollTo(0, 0);")

page_html = driver.page_source
pageSoup = bs4.BeautifulSoup(page_html, 'html.parser')
containers = pageSoup.findAll('div', {'class':"isv-r PNCib MSM1fd BUooTd"} )

print(len(containers))

len_containers = len(containers)

for i in range(1, len_containers+1):
    if i % 25 == 0:
        continue

    xPath = """//*[@id="islrg"]/div[1]/div[%s]"""%(i)

    previewImageXPath = """//*[@id="islrg"]/div[1]/div[%s]/a[1]/div[1]/img"""%(i)
    previewImageElement = driver.find_element_by_xpath(previewImageXPath)
    previewImageURL = previewImageElement.get_attribute("src")
    #print("preview URL", previewImageURL)


    #print(xPath)


    driver.find_element_by_xpath(xPath).click()
    #time.sleep(3)

    #//*[@id="islrg"]/div[1]/div[16]/a[1]/div[1]/img

    #input('waawgawg another wait')

    # page = driver.page_source
    # soup = bs4.BeautifulSoup(page, 'html.parser')
    # ImgTags = soup.findAll('img', {'class': 'n3VNCb', 'jsname': 'HiaYvf', 'data-noaft': '1'})
    # print("number of the ROI tags", len(ImgTags))
    # link = ImgTags[1].get('src')
    # #print(len(ImgTags))
    # #print(link)
    #
    # n=0
    # for tag in ImgTags:
    #     print(n, tag)
    #     n+=1
    # print(len(ImgTags))

    #/html/body/div[2]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img

    #It's all about the wait

    timeStarted = time.time()
    while True:

        imageElement = driver.find_element_by_xpath("""//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img""")
        imageURL= imageElement.get_attribute('src')

        if imageURL != previewImageURL:
            #print("actual URL", imageURL)
            break

        else:
            #making a timeout if the full res image can't be loaded
            currentTime = time.time()

            if currentTime - timeStarted > 10:
                print("Timeout! Will download a lower resolution image and move onto the next one")
                break


    #Downloading image
    try:
        download_image(imageURL, folder_name, i)
        print("Downloaded element %s out of %s total. URL: %s" % (i, len_containers + 1, imageURL))
    except:
        print("Couldn't download an image %s, continuing downloading the next one"%(i))

WebDriverException: Message: 'C:\Users\net51\Documents\MyPythonScripts\Drivers\chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [49]:
from webdriver_manager.chrome import ChromeDriverManager

In [56]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

TypeError: __init__() got an unexpected keyword argument 'service'

In [57]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
#service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")
#service = Service(executable_path="/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service)

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)


In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.service import Service
from selenium import webdriver
#service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")
#service = Service(executable_path="/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service)

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
d = webdriver.Chrome('/home/<user>/chromedriver',chrome_options=chrome_options)
d.get('https://www.google.nl/')

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)


In [59]:
import time

from selenium import webdriver



driver = webdriver.Chrome('/usr/local/bin/chromedriver')

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)


In [61]:
#Selenium helps you use this executable to automate Chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#service = Service('/usr/local/bin/chromedriver')
#print(service)

from multiprocessing.sharedctypes import Value
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from datetime import datetime as dt
from PIL import Image
import time
import os

# Download the driver from chromedriver website for relevant OS i.e. MAC, Windows, Debian, etc.
PATH = '/usr/local/bin/chromedriver'
wd = webdriver.Chrome(executable_path=PATH)

#wd = Service('/usr/local/bin/chromedriver')
# wd = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# wd.maximize_window()
def get_images_from_google(wd, delay, max_images, url):
	def scroll_down(wd):
		wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(delay)

	url = url
	wd.get(url)

	image_urls = set()
	skips = 0
	while len(image_urls) + skips < max_images:
		scroll_down(wd)
		thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")

		for img in thumbnails[len(image_urls) + skips:max_images]:
			try:
				img.click()
				time.sleep(delay)
			except:
				continue

			images = wd.find_elements(By.CLASS_NAME, "n3VNCb")
			for image in images:
				if image.get_attribute('src') in image_urls:
					max_images += 1
					skips += 1
					break

				if image.get_attribute('src') and 'http' in image.get_attribute('src'):
					image_urls.add(image.get_attribute('src'))
					##print(f"Found {len(image_urls)}")

	return image_urls


def download_image(down_path, url, file_name, image_type='JPEG',
                   verbose=True):
    try:
        time = dt.now()
        curr_time = time.strftime('%H:%M:%S')
        #Content of the image will be a url
        img_content = requests.get(url).content
        #Get the bytes IO of the image
        img_file = io.BytesIO(img_content)
        #Stores the file in memory and convert to image file using Pillow
        image = Image.open(img_file)
        file_pth = down_path + file_name

        with open(file_pth, 'wb') as file:
            image.save(file, image_type)

        if verbose == True:
            print(f'The image: {file_pth} downloaded successfully at {curr_time}.')
    except Exception as e:
        print(f'Unable to download image from Google Photos due to\n: {str(e)}')



if __name__ == '__main__':
    # Google search URLS
    google_urls = [
                   'https://www.google.com/search?q=brice+samba&tbm=isch&ved=2ahUKEwinwYCZ3cj3AhVOyRoKHVjuC5UQ2-cCegQIABAA&oq=brice+samba&gs_lcp=CgNpbWcQAzIHCCMQ7wMQJzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBAgAEB4yBAgAEBgyBAgAEBgyBAgAEBgyBAgAEBg6CAgAELEDEIMBOggIABCABBCxAzoHCAAQsQMQQzoECAAQQzoECAAQAzoGCAAQCBAeULQIWOYXYP4ZaABwAHgAgAFKiAGBBpIBAjEymAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=V_RzYue5Ls6Sa9jcr6gJ&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=keinan+davis+footballer&tbm=isch&ved=2ahUKEwjn7dqj3cj3AhUBdBoKHXxWCAMQ2-cCegQIABAA&oq=keinan+davis+footballer&gs_lcp=CgNpbWcQAzIECAAQGDoHCCMQ7wMQJzoFCAAQgAQ6BAgAEB46CAgAELEDEIMBOgsIABCABBCxAxCDAToICAAQgAQQsQM6BAgAEEM6BwgAELEDEEM6BggAEAUQHjoGCAAQCBAeUO8JWO4qYOAraABwAHgAgAFViAH8C5IBAjI0mAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=bvRzYuf-DIHoafysoRg&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=ethan+horvath+football&tbm=isch&ved=2ahUKEwixt9HN3cj3AhVJ3RoKHdoiAeoQ2-cCegQIABAA&oq=ethan+horvath+football&gs_lcp=CgNpbWcQAzoHCCMQ7wMQJzoECAAQQzoFCAAQgAQ6BAgAEB46BAgAEBhQiQRYgQ9gxRBoAHAAeACAAVeIAeUEkgECMTCYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=xvRzYrG8CMm6a9rFhNAO&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=steve+cook+forest&tbm=isch&ved=2ahUKEwiXipXR3cj3AhVKexoKHYg2BjcQ2-cCegQIABAA&oq=steve+cook+&gs_lcp=CgNpbWcQARgAMgcIIxDvAxAnMgQIABADMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQ6CAgAEIAEELEDOggIABCxAxCDAToECAAQQzoLCAAQgAQQsQMQgwFQ3gZYhhJgkSJoAHAAeACAAbMBiAGFB5IBBDExLjGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=zfRzYtfvIMr2aYjtmLgD&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=joe+worrall&tbm=isch&ved=2ahUKEwiLhJna3cj3AhXW44UKHYvcCjIQ2-cCegQIABAA&oq=joe+worrall&gs_lcp=CgNpbWcQAzIHCCMQ7wMQJzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBAgAEEMyBQgAEIAEMgUIABCABDIECAAQGDIECAAQGDoICAAQsQMQgwE6CwgAEIAEELEDEIMBOggIABCABBCxAzoKCAAQsQMQgwEQQzoHCAAQsQMQQ1DcZ1jXdWDPeWgBcAB4AIABkAGIAY8HkgEEMTEuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=4PRzYouUHdbHlwSLuauQAw&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=ryan+yates&tbm=isch&ved=2ahUKEwje3vni3cj3AhXa0oUKHfW4D5kQ2-cCegQIABAA&oq=ryan+yates&gs_lcp=CgNpbWcQAzIHCCMQ7wMQJzIFCAAQgAQyBQgAEIAEMgYIABAFEB4yBAgAEBgyBAgAEBgyBAgAEBgyBAgAEBgyBAgAEBgyBAgAEBg6BAgAEEM6CAgAELEDEIMBOgsIABCABBCxAxCDAToICAAQgAQQsQM6CggjEO8DEOoCECc6CggAELEDEIMBEEM6BwgAELEDEENQpghYlBdgwxhoAXAAeACAAViIAbgGkgECMTKYAQCgAQGqAQtnd3Mtd2l6LWltZ7ABCsABAQ&sclient=img&ei=8vRzYt6dM9qllwT18b7ICQ&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=jack+colback+footballer&tbm=isch&ved=2ahUKEwiGi83s3cj3AhUDpBoKHTQKDzcQ2-cCegQIABAA&oq=jack+colback+footballer&gs_lcp=CgNpbWcQAzIECAAQGDoHCCMQ7wMQJzoFCAAQgAQ6BggAEAcQHjoECAAQQzoICAAQsQMQgwE6CwgAEIAEELEDEIMBOggIABCABBCxAzoGCAAQBRAeOgQIABAeOgYIABAIEB5QowhY0iRgpSVoAHAAeACAAZgBiAGxDZIBBDIzLjGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=B_VzYsbrBIPIarSUvLgD&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=scott+mckenna&tbm=isch&ved=2ahUKEwj-mJD53cj3AhUQQhoKHQpnDfMQ2-cCegQIABAA&oq=scott+&gs_lcp=CgNpbWcQARgAMgcIIxDvAxAnMgQIABBDMgcIABCxAxBDMgQIABBDMgQIABBDMgcIABCxAxBDMggIABCxAxCDATILCAAQgAQQsQMQgwEyCAgAEIAEELEDMgsIABCABBCxAxCDAToECAAQAzoFCAAQgARQ6QRYwgtghxZoAHAAeACAAUuIAd0DkgEBN5gBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=IfVzYr6EFZCEaYrOtZgP&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=cafu+forest&tbm=isch&ved=2ahUKEwi7xdeN3sj3AhUE4hoKHcHTCO8Q2-cCegQIABAA&oq=cafu+forest&gs_lcp=CgNpbWcQAzIFCAAQgAQyBggAEAgQHjoECAAQQzoECAAQHjoECAAQGFAAWKIHYKUJaABwAHgAgAFZiAHSA5IBATeYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=TPVzYrvvG4TEa8Gno_gO&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=max+lowe+footballer&tbm=isch&ved=2ahUKEwjpj4aQ3sj3AhUL_BoKHSF6CmwQ2-cCegQIABAA&oq=max+lowe+footballer&gs_lcp=CgNpbWcQAzIECAAQGDoHCCMQ7wMQJzoGCAAQCBAeOgsIABCABBCxAxCDAToFCAAQgAQ6BAgAEAM6CAgAELEDEIMBOggIABCABBCxAzoECAAQQzoHCAAQsQMQQzoGCAAQBRAeOgQIABAeUKMIWLsiYJgjaABwAHgAgAGdAYgB9wqSAQQxOS4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=UfVzYqmjGYv4a6H0qeAG&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924',
                   'https://www.google.com/search?q=joe+lolley&rlz=1C1CHBF_en-GBGB924GB924&hl=en&sxsrf=ALiCzsYdA_zTQwIGhtfmL__WXkmGyvR7aQ:1651766657726&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjduIqn3sj3AhULJMAKHZ8hCoYQ_AUoAnoECAEQBA&biw=1920&bih=969&dpr=1',
                   'https://www.google.com/search?q=alex+mighten&tbm=isch&ved=2ahUKEwi73fyn3sj3AhUD-4UKHSjyBpAQ2-cCegQIABAA&oq=alex&gs_lcp=CgNpbWcQARgAMgcIIxDvAxAnMgcIIxDvAxAnMgoIABCxAxCDARBDMgQIABBDMgQIABBDMgQIABBDMggIABCABBCxAzIECAAQQzIICAAQgAQQsQMyCAgAEIAEELEDOgUIABCABDoECAAQGDoLCAAQgAQQsQMQgwE6CAgAELEDEIMBOgcIABCxAxBDUKIHWJwLYLwUaABwAHgAgAFLiAHIApIBATWYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=g_VzYvuPJIP2lwSo5JuACQ&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924&hl=en'
    ]
    
    # Labels for the players
    labels = [
        'brice_samba','keinan_davis', 'ethan_horvath', 'steve_cook', 'joe_worrall',
        'ryan_yates', 'jack_colback', 'scott_mckenna', 'cafu', 'max_lowe',
        'joe_lolley', 'alex_mighten'
    ]

    # Check the length of the lists
    if len(google_urls) != len(labels):
        raise ValueError('The length of the url list does not match the labels list.')

    player_path = 'images/nottingham_forest/'
    # Make the directory if it doesn't exist
    for lbl in labels:
        if not os.path.exists(player_path + lbl):
            print(f'Making directory: {str(lbl)}')
            os.makedirs(player_path+lbl)

    for url_current, lbl in zip(google_urls, labels):
        urls = get_images_from_google(wd, 0, 100, url_current)
        # Once we have added our urls to empty set then 
        for i, url in enumerate(urls):
            download_image(down_path=f'images/nottingham_forest/{lbl}/', 
                        url=url, 
                        file_name=str(i+1)+ '.jpg',
                        verbose=True) 
    wd.quit()

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)


In [46]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
service = Service('/usr/local/bin/chromedriver')
print(service)

In [ ]:
import os
import time
import json

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def download_google_images(search_query: str, number_of_images=500) -> None:
    '''Download google images with this function\n
       Takes -> search_query, number_of_images\n
       Returns -> None
    '''
    def scroll_to_bottom():
        '''Scroll to the bottom of the page
        '''
        last_height = driver.execute_script('return document.body.scrollHeight')
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(2)

            new_height = driver.execute_script('return document.body.scrollHeight')
            try:
                element = driver.find_element(
                    by=By.CSS_SELECTOR,
                    value='.YstHxe input'
                )
                element.click()
                time.sleep(2)
            except:
                pass

            if new_height == last_height:
                break

            last_height = new_height

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    url = 'https://images.google.com/'

    driver.get(
        url=url
    )

    box = driver.find_element(
        by=By.XPATH,
        value="//input[contains(@class,'gLFyf gsfi')]"
    )

    box.send_keys(search_query)
    box.send_keys(Keys.ENTER)
    time.sleep(2)

    scroll_to_bottom()
    time.sleep(2)

    img_results = driver.find_elements(
        by=By.XPATH,
        value="//img[contains(@class,'rg_i Q4LuWd')]"
    )

    print(f'Totla images -> {len(img_results)}')

    count = 0

    for img_result in img_results:
        try:
            WebDriverWait(
                driver,
                15
            ).until(
                EC.element_to_be_clickable(
                    img_result
                )
            )
            img_result.click()
            time.sleep(2)

            actual_img = driver.find_element(
                by=By.XPATH,
                value="//img[contains(@class,'n3VNCb')]"
            )

            src = actual_img.get_attribute('src')
            width = actual_img.get_attribute('width')
            height = actual_img.get_attribute('height')

            if 'https://' in src:
                print(src)
                print(width)
                print(height)
            else:
                print('Base 64 in source.')
                print(width)
                print(height)
        except ElementClickInterceptedException as e:
            print(e)
            print('Image is not clickable.')
        
        count += 1
        if count == number_of_images:
            break


cwd = os.getcwd()

tags = open(f'{cwd}/playground/tags.json')
tags = json.load(tags)

tags = tags['tags']

for tag in tags:
    download_google_images(tag, 10)

In [10]:
import PIL

In [12]:
import sys
from PIL import Image
import io
import urllib.request
import urllib.parse
import requests

BASE_URI = "https://collectionapi.metmuseum.org/public/collection/v1/"

def emo_search(query):
    '''Look for a given work based on user query'''
    url = BASE_URI + "search?q=" + query
    response = requests.get(url).json()
    artwork_lst = response['objectIDs']
    for i, item in enumerate(artwork_lst):
        art_url = BASE_URI + "objects/" + str(item)
        art_response = requests.get(art_url).json()
        print(art_response)
#         if art_response['constituents'] is not None and art_response['primaryImage']:
#             print(f"{art_response['title']} by : {art_response['constituents'][0]['name']} at {art_response['primaryImage']}")
#             img_response = requests.get('https://picsum.photos/seed/picsum/200/300')
#             in_memory_file = io.BytesIO(img_response.content)
#             im = Image.open(in_memory_file)
#             im.show()

def main():
    '''Ask user for a keyword to be used as query'''
    query = input("Show me\n> ")
    emo = emo_search(query)

if __name__ == '__main__':
    try:
        while True:
            main()
    except KeyboardInterrupt:
        print('\nGoodbye!')
        sys.exit(0)

Show me
> aeroplane
{'objectID': 480671, 'isHighlight': False, 'accessionNumber': '19.170', 'accessionYear': '1919', 'isPublicDomain': False, 'primaryImage': '', 'primaryImageSmall': '', 'additionalImages': [], 'constituents': [{'constituentID': 144941, 'role': 'Designer', 'name': 'Julio Kilenyi', 'constituentULAN_URL': 'http://vocab.getty.edu/page/ulan/500049066', 'constituentWikidata_URL': 'https://www.wikidata.org/wiki/Q15999333', 'gender': ''}], 'department': 'Modern and Contemporary Art', 'objectName': 'Medal', 'title': 'First Trans-Atlantic Flight, 1919', 'culture': '', 'period': '', 'dynasty': '', 'reign': '', 'portfolio': '', 'artistRole': 'Designer', 'artistPrefix': '', 'artistDisplayName': 'Julio Kilenyi', 'artistDisplayBio': 'American, 1885–1959', 'artistSuffix': '', 'artistAlphaSort': 'Kilenyi, Julio', 'artistNationality': 'American', 'artistBeginDate': '1885', 'artistEndDate': '1959', 'artistGender': '', 'artistWikidata_URL': 'https://www.wikidata.org/wiki/Q15999333', 'art

{'objectID': 634823, 'isHighlight': False, 'accessionNumber': '63.350.246.121.113', 'accessionYear': '1963', 'isPublicDomain': False, 'primaryImage': '', 'primaryImageSmall': '', 'additionalImages': [], 'constituents': [{'constituentID': 104502, 'role': 'Publisher', 'name': 'Issued by American Tobacco Company', 'constituentULAN_URL': '(not assigned)', 'constituentWikidata_URL': 'https://www.wikidata.org/wiki/Q467075', 'gender': ''}, {'constituentID': 164155, 'role': 'Artist', 'name': 'Original photograph by Underwood &amp; Underwood', 'constituentULAN_URL': 'http://vocab.getty.edu/page/ulan/500033291', 'constituentWikidata_URL': 'https://www.wikidata.org/wiki/Q2492951', 'gender': ''}], 'department': 'Drawings and Prints', 'objectName': 'Print', 'title': 'Card No. 108, Aeroplane (French) Being Fired Upon by Germans, from the World War I Scenes series (T121) issued by Sweet Caporal Cigarettes', 'culture': '', 'period': '', 'dynasty': '', 'reign': '', 'portfolio': '', 'artistRole': 'Publi

AssertionError: 

In [14]:
import os
import time
import json

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def download_google_images(search_query: str, number_of_images=500) -> None:
    '''Download google images with this function\n
       Takes -> search_query, number_of_images\n
       Returns -> None
    '''
    def scroll_to_bottom():
        '''Scroll to the bottom of the page
        '''
        last_height = driver.execute_script('return document.body.scrollHeight')
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(2)

            new_height = driver.execute_script('return document.body.scrollHeight')
            try:
                element = driver.find_element(
                    by=By.CSS_SELECTOR,
                    value='.YstHxe input'
                )
                element.click()
                time.sleep(2)
            except:
                pass

            if new_height == last_height:
                break

            last_height = new_height

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    url = 'https://images.google.com/'

    driver.get(
        url=url
    )

    box = driver.find_element(
        by=By.XPATH,
        value="//input[contains(@class,'gLFyf gsfi')]"
    )

    box.send_keys(search_query)
    box.send_keys(Keys.ENTER)
    time.sleep(2)

    scroll_to_bottom()
    time.sleep(2)

    img_results = driver.find_elements(
        by=By.XPATH,
        value="//img[contains(@class,'rg_i Q4LuWd')]"
    )

    print(f'Totla images -> {len(img_results)}')

    count = 0

    for img_result in img_results:
        try:
            WebDriverWait(
                driver,
                15
            ).until(
                EC.element_to_be_clickable(
                    img_result
                )
            )
            img_result.click()
            time.sleep(2)

            actual_img = driver.find_element(
                by=By.XPATH,
                value="//img[contains(@class,'n3VNCb')]"
            )

            src = actual_img.get_attribute('src')
            width = actual_img.get_attribute('width')
            height = actual_img.get_attribute('height')

            if 'https://' in src:
                print(src)
                print(width)
                print(height)
            else:
                print('Base 64 in source.')
                print(width)
                print(height)
        except ElementClickInterceptedException as e:
            print(e)
            print('Image is not clickable.')
        
        count += 1
        if count == number_of_images:
            break


# cwd = os.getcwd()

# tags = open(f'{cwd}/playground/tags.json')
# tags = json.load(tags)

# tags = tags['tags']

# for tag in tags:
#     download_google_images(tag, 10)

In [16]:
download_google_images("boat painting", 10)

TypeError: __init__() got an unexpected keyword argument 'service'

In [17]:
from art_api import config
config.CLASSES

['aeroplane',
 'bird',
 'boat',
 'chair',
 'cow',
 'diningtable',
 'dog',
 'horse',
 'sheep',
 'train']